In [1]:
import time
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.ndimage.measurements import label
from feature_extraction import *
import glob

## Settings

In [10]:
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

colorspace = 'YCrCb' # RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial = 32 # Spatial binning dimensions
histbin = 32 # Number of histogram bins

## Reading dataset
The GTI dataset is splited manually

In [3]:
cars_gti_train = glob.glob('../vehicles/GTI*/*.png')
cars_gti_test = glob.glob('../vehicles/test_GTI*/*.png')
cars_kitti = glob.glob('../vehicles/KITTI_extracted/*.png')
notcars_gti = glob.glob('../non-vehicles/GTI/*.png')
notcars_extra = glob.glob('../non-vehicles/Extras/*.png')

print('GTI cars train dataset number: %d' % len(cars_gti_train))
print('GTI cars test dataset number: %d' % len(cars_gti_test))
print('KITTI cars dataset number: %d' % len(cars_kitti))
print('GTI notcars dataset number: %d' % len(notcars_gti))
print('Extra notcars dataset number: %d' % len(notcars_extra))


import random
def random_split_list(data, test_size = 0.1):
    random.shuffle(data)
    data_test_num = int(test_size * len(data))
    data_train = data[data_test_num:]
    data_test = data[:data_test_num]
    print('train size: %d' % len(data_train))
    print('test size: %d' % len(data_test))
    return data_train, data_test
    
notcars = notcars_gti + notcars_extra

test_size = 0.1
notcars_train, notcars_test = random_split_list(notcars, test_size) 
cars_kitti_train, cars_kitti_test = random_split_list(cars_kitti, test_size)

cars_train = cars_gti_train + cars_kitti_train
cars_test = cars_gti_test + cars_kitti_test

GTI cars train dataset number: 2521
GTI cars test dataset number: 305
KITTI cars dataset number: 5966
GTI notcars dataset number: 3900
Extra notcars dataset number: 5068
train size: 8072
test size: 896
train size: 5370
test size: 596


In [4]:
from sklearn.utils import shuffle
def train_model(cars_train, notcars_train, cars_test, notcars_test):
    t = time.time()
    car_train_features = extract_features(cars_train, color_space = colorspace,
                                    spatial_size = (spatial, spatial),
                                    hist_bins = histbin, orient = orient,
                                    pix_per_cell = pix_per_cell,
                                    cell_per_block = cell_per_block,
                                    hog_channel = hog_channel,
                                    spatial_feat = spatial_feat,
                                    hist_feat = hist_feat,
                                    hog_feat = hog_feat)
    
    notcar_train_features = extract_features(notcars_train, color_space = colorspace,
                                    spatial_size = (spatial, spatial),
                                    hist_bins = histbin, orient = orient,
                                    pix_per_cell = pix_per_cell,
                                    cell_per_block = cell_per_block,
                                    hog_channel = hog_channel,
                                    spatial_feat = spatial_feat,
                                    hist_feat = hist_feat,
                                    hog_feat = hog_feat)
    
    car_test_features = extract_features(cars_test, color_space = colorspace,
                                    spatial_size = (spatial, spatial),
                                    hist_bins = histbin, orient = orient,
                                    pix_per_cell = pix_per_cell,
                                    cell_per_block = cell_per_block,
                                    hog_channel = hog_channel,
                                    spatial_feat = spatial_feat,
                                    hist_feat = hist_feat,
                                    hog_feat = hog_feat)
    
    notcar_test_features = extract_features(notcars_test, color_space = colorspace,
                                    spatial_size = (spatial, spatial),
                                    hist_bins = histbin, orient = orient,
                                    pix_per_cell = pix_per_cell,
                                    cell_per_block = cell_per_block,
                                    hog_channel = hog_channel,
                                    spatial_feat = spatial_feat,
                                    hist_feat = hist_feat,
                                    hog_feat = hog_feat)
    
    t1 = time.time()
    print(round(t1-t, 2), 'Seconds to extract features ...')
    
    X_train = np.vstack((car_train_features, notcar_train_features)).astype(np.float64)
    X_test = np.vstack((car_test_features, notcar_test_features)).astype(np.float64)
    X_scaler = StandardScaler().fit(X_train)
    X_train = X_scaler.transform(X_train)
    X_test = X_scaler.transform(X_test)
    y_train = np.hstack((np.ones(len(car_train_features)), np.zeros(len(notcar_train_features))))
    y_test = np.hstack((np.ones(len(car_test_features)), np.zeros(len(notcar_test_features))))
    
    rand_state = np.random.randint(0,100)
    X_train, y_train = shuffle(X_train, y_train, random_state = rand_state)
    X_test, y_test = shuffle(X_test, y_test, random_state = rand_state)
    
    
    print("Train dataset size: %d" % len(X_train))
    print("Test dataset size: %d" % len(X_test))
    
    print('Color space:', colorspace, 'Spatial:', spatial, 'histbins:', histbin,
          'Using:',orient,'orientations',pix_per_cell,
          'pixels per cell and', cell_per_block,'cells per block')
    print('Feature vector length:', len(X_train[0]))
    
    svc = LinearSVC()
    t = time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVC ...')
    print('Train Accuracy of SVC = ', round(svc.score(X_train, y_train), 4))
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    return svc, X_scaler

In [11]:
trained_model, X_scaler = train_model(cars_train, notcars_train, cars_test, notcars_test)

84.18 Seconds to extract features ...
Train dataset size: 15963
Test dataset size: 1797
Color space: YCrCb Spatial: 32 histbins: 32 Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
28.36 Seconds to train SVC ...
Train Accuracy of SVC =  1.0
Test Accuracy of SVC =  0.9889


In [12]:
data = {
    'svc': trained_model,
    'colorspace' : colorspace,
    'orient' : orient,
    'pix_per_cell' : pix_per_cell,
    'cell_per_block' : cell_per_block,
    'hog_channel' : hog_channel,
    'spatial' : spatial,
    'histbin' : histbin,
    'X_scaler' : X_scaler
    }

with open('trained_svc_model.p', 'wb') as pickle_file:
    pickle.dump(data, pickle_file)